In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import scipy
from skimage import io
import multiprocessing as mp
from itertools import repeat

In [2]:
datagen = ImageDataGenerator(
    rotation_range=20, #rotate between 0 and 20 degree
    width_shift_range=0.1, # shift width
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest', # constant, nearest, reflect
    cval=125,
    brightness_range=[0.7,1.0])



In [3]:
dir = 'downscaledImg/'
augmentedPath = 'augmentedImg/'

# how many images should be produced per source image
iterations = 5;

# read all classes
classes = os.listdir(dir)

In [4]:
# process an image
def processImage(srcDir, cls, img, targetPath):
    i = 0
    try:
        path = srcDir+cls+'/'+img
        print(path)
        x = io.imread(path)
        x = x.reshape((1, ) + x.shape)
        for batch in datagen.flow(x, batch_size=16,
                          save_to_dir=targetPath,
                          save_prefix='aug-'+img,
                          save_format='png'):
            i+=1
            if i >= iterations: break;
    except BaseException as err:
        print(type(err))
        if(type(err) == KeyboardInterrupt):
            raise Exception("stopped process")
        print(f"Unexpected {err=}, {type(err)=}")

def processImageClass(cls, subThreads):
    print("starting with: "+cls)
    pool = mp.Pool(subThreads)
    targetPath = augmentedPath+cls
    images = os.listdir(dir+cls)
    if not(os.path.exists(targetPath)): os.mkdir(targetPath)
    #for img in images:
    pool.starmap(processImage, zip(repeat(dir), repeat(cls), images, repeat(targetPath)))
    #pool.starmap(processImage, [dir, cls, (images), targetPath])
        #processImage(dir, cls, img, 'hello', targetPath)
    pool.close()
            
    print("done with: "+cls)

In [5]:
#define pool for parallel processing
processors = mp.cpu_count()
if not(os.path.exists(augmentedPath)): os.mkdir(augmentedPath)
for cls in classes:
    if(os.path.isdir(dir+cls)):
        processImageClass(cls, processors)

print('DONE!!!')


starting with: .ipynb_checkpoints
done with: .ipynb_checkpoints
starting with: Amrit
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.10.jpeg
downscaledImg/Amrit/Capture.PNG
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.09.jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.19 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.12 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.16 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.18 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.20 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.16.jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.13 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.14.jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.17 (2).jpeg
downscaledImg/Amrit/ssssssss.PNG
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.21 (1).jpeg
downscaledImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.12.jpeg

downscaledImg/nasir/FB_IMG_1496745814204.jpg
downscaledImg/nasir/IMG_20170904_110732.jpg
downscaledImg/nasir/IMG_20170904_191024.jpg
downscaledImg/nasir/FB_IMG_1496745804610.jpg
downscaledImg/nasir/IMG_20171021_144933.jpg
downscaledImg/nasir/IMG_20171019_191910.jpg
downscaledImg/nasir/IMG_20171104_152615.jpg
downscaledImg/nasir/IMG_20171223_105805.jpg
downscaledImg/nasir/IMG_20171229_070906.jpg
downscaledImg/nasir/IMG_20180107_153615.jpg
downscaledImg/nasir/IMG_20180213_192948.jpg
downscaledImg/nasir/IMG_20180228_115531.jpg
downscaledImg/nasir/IMG_20180228_145504.jpg
downscaledImg/nasir/IMG_20180306_190909.jpg
downscaledImg/nasir/IMG_20180718_054547.jpg
downscaledImg/nasir/IMG_2452.JPG
downscaledImg/nasir/IMG_2598.JPG
downscaledImg/nasir/IMG_2809.JPG
downscaledImg/nasir/IMG_2810.JPG
downscaledImg/nasir/IMG_2811.JPG
downscaledImg/nasir/JPEG_20180513_195608_1470689297.jpg
downscaledImg/nasir/nkn.jpg
downscaledImg/nasir/photo_art_20180228_145150.png
done with: nasir
starting with: noman
d